In [1]:
# Install necessary libraries
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install alpha_vantage

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Install Selenium and ChromeDriver
!pip install selenium-wire
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 36.2 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRele

In [5]:
from seleniumwire import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Configure Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize Selenium Wire driver
browser = webdriver.Chrome(options=chrome_options)

# Navigate to the S&P 500 stock list page
url = "https://stockanalysis.com/list/sp-500-stocks/"
browser.get(url)

# Wait for the table to be present on the page (up to 10 seconds)
try:
    # Use XPath to locate the table
    table = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH, "//table"))
    )
    print("Table found!")
except Exception as e:
    print(f"Error: {e}")
    browser.quit()

# Extract rows from the table using XPath to locate 'tr' elements
rows = table.find_elements(By.XPATH, ".//tr")

# Initialize lists to store stock tickers and other data
tickers = []
company_names = []
market_caps = []
stock_prices = []
percent_changes = []
revenues = []

# Loop through the rows and extract stock information
for row in rows[1:101]:  # Limit to top 100 rows
    cols = row.find_elements(By.TAG_NAME, 'td')
    if len(cols) > 0:
        tickers.append(cols[1].text)  # Ticker is in the 2nd column
        company_names.append(cols[2].text)  # Company name is in the 3rd column
        market_caps.append(cols[3].text)  # Market cap is in the 4th column
        stock_prices.append(cols[4].text)  # Stock price is in the 5th column
        percent_changes.append(cols[5].text)  # % Change is in the 6th column
        revenues.append(cols[6].text)  # Revenue is in the 7th column

# Close the browser
browser.quit()

# Create a DataFrame from the collected data
top_100_stocks = pd.DataFrame({
    'Ticker': tickers,
    'Company Name': company_names,
    'Market Cap': market_caps,
    'Stock Price': stock_prices,
    '% Change': percent_changes,
    'Revenue': revenues
})

# Save the DataFrame to a CSV file
csv_path = '/content/drive/My Drive/top_100_sp500_stocks.csv'
top_100_stocks.to_csv(csv_path, index=False)

print(f"Top 100 stocks have been saved to {csv_path}.")

Table found!
Top 100 stocks have been saved to /content/drive/My Drive/top_100_sp500_stocks.csv.


In [6]:
# Install system dependencies and ta-lib
!pip install yfinance pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=4ebec0c7e0421beefa652e0bba5cb4bf2c4ed1c1b414bc0cf7c1912614ad21d3
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [7]:
# Load the list of top 100 stock tickers from Google Drive
csv_path = '/content/drive/My Drive/top_100_sp500_stocks.csv'
top_100_stocks = pd.read_csv(csv_path)

In [8]:
# Display the list of stock tickers
print(top_100_stocks.head())

  Ticker           Company Name Market Cap Stock Price % Change  Revenue
0   AAPL             Apple Inc.  3,369.68B      222.91   -1.33%  391.04B
1   NVDA     NVIDIA Corporation  3,321.36B      135.40    1.99%   96.31B
2   MSFT  Microsoft Corporation  3,051.05B      410.37    0.99%  254.19B
3   GOOG          Alphabet Inc.  2,104.29B      172.65   -0.02%  339.86B
4  GOOGL          Alphabet Inc.  2,096.76B      171.29    0.11%  339.86B


In [14]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

# List of stock tickers
tickers = top_100_stocks['Ticker'].tolist()

# Initialize an empty DataFrame to store the combined data
all_stock_data = pd.DataFrame()

# Loop through each ticker to download data and calculate indicators
for ticker in tickers:
    print(f"\nProcessing data for {ticker}...")

    # Download historical data for the past year
    stock_data = yf.download(ticker, period="1y", interval="1d")

    # Check if data is empty
    if stock_data.empty:
        print(f"No data for {ticker}")
        continue

    # Print the column names for diagnosis
    print(f"Original columns for {ticker}: {stock_data.columns}")

    # Flatten column names if they are multi-index
    if isinstance(stock_data.columns, pd.MultiIndex):
        stock_data.columns = stock_data.columns.get_level_values(1)
        print(f"Flattened columns for {ticker}: {stock_data.columns}")

    # Rename columns if standard columns are missing
    # This will standardize the names if they have the ticker symbol as part of them
    stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

    # Verify that the required columns are present
    if 'Close' not in stock_data.columns:
        print(f"Expected column 'Close' not found in {ticker} data. Skipping.")
        continue

    # Reset the index to ensure it is a standard DataFrame
    stock_data.reset_index(inplace=True)

    # Calculate technical indicators using pandas_ta and append to the DataFrame
    stock_data.ta.sma(length=50, append=True)         # 50-day Simple Moving Average
    stock_data.ta.ema(length=50, append=True)         # 50-day Exponential Moving Average
    stock_data.ta.rsi(length=14, append=True)         # 14-day Relative Strength Index
    stock_data.ta.macd(append=True)                   # MACD
    stock_data.ta.bbands(length=20, append=True)      # Bollinger Bands

    # Add a column for the stock ticker
    stock_data['Ticker'] = ticker

    # Concatenate the data for this ticker with the main DataFrame
    all_stock_data = pd.concat([all_stock_data, stock_data])

# Save the combined data to Google Drive
combined_csv_path = '/content/drive/My Drive/top_100_sp500_yfinance_data.csv'
all_stock_data.to_csv(combined_csv_path, index=False)

print(f"\nAll stock data with technical indicators saved to {combined_csv_path}.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Processing data for AAPL...
Original columns for AAPL: MultiIndex([('Adj Close', 'AAPL'),
            (    'Close', 'AAPL'),
            (     'High', 'AAPL'),
            (      'Low', 'AAPL'),
            (     'Open', 'AAPL'),
            (   'Volume', 'AAPL')],
           names=['Price', 'Ticker'])
Flattened columns for AAPL: Index(['AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL'], dtype='object', name='Ticker')

Processing data for NVDA...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Original columns for NVDA: MultiIndex([('Adj Close', 'NVDA'),
            (    'Close', 'NVDA'),
            (     'High', 'NVDA'),
            (      'Low', 'NVDA'),
            (     'Open', 'NVDA'),
            (   'Volume', 'NVDA')],
           names=['Price', 'Ticker'])
Flattened columns for NVDA: Index(['NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA'], dtype='object', name='Ticker')

Processing data for MSFT...
Original columns for MSFT: MultiIndex([('Adj Close', 'MSFT'),
            (    'Close', 'MSFT'),
            (     'High', 'MSFT'),
            (      'Low', 'MSFT'),
            (     'Open', 'MSFT'),
            (   'Volume', 'MSFT')],
           names=['Price', 'Ticker'])
Flattened columns for MSFT: Index(['MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT'], dtype='object', name='Ticker')

Processing data for GOOG...
Original columns for GOOG: MultiIndex([('Adj Close', 'GOOG'),
            (    'Close', 'GOOG'),
            (     'High', 'GOOG'),
            (      'Low', 'G

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

Original columns for DE: MultiIndex([('Adj Close', 'DE'),
            (    'Close', 'DE'),
            (     'High', 'DE'),
            (      'Low', 'DE'),
            (     'Open', 'DE'),
            (   'Volume', 'DE')],
           names=['Price', 'Ticker'])
Flattened columns for DE: Index(['DE', 'DE', 'DE', 'DE', 'DE', 'DE'], dtype='object', name='Ticker')

Processing data for MMC...
Original columns for MMC: MultiIndex([('Adj Close', 'MMC'),
            (    'Close', 'MMC'),
            (     'High', 'MMC'),
            (      'Low', 'MMC'),
            (     'Open', 'MMC'),
            (   'Volume', 'MMC')],
           names=['Price', 'Ticker'])
Flattened columns for MMC: Index(['MMC', 'MMC', 'MMC', 'MMC', 'MMC', 'MMC'], dtype='object', name='Ticker')

Processing data for PLD...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Original columns for PLD: MultiIndex([('Adj Close', 'PLD'),
            (    'Close', 'PLD'),
            (     'High', 'PLD'),
            (      'Low', 'PLD'),
            (     'Open', 'PLD'),
            (   'Volume', 'PLD')],
           names=['Price', 'Ticker'])
Flattened columns for PLD: Index(['PLD', 'PLD', 'PLD', 'PLD', 'PLD', 'PLD'], dtype='object', name='Ticker')

Processing data for INTC...
Original columns for INTC: MultiIndex([('Adj Close', 'INTC'),
            (    'Close', 'INTC'),
            (     'High', 'INTC'),
            (      'Low', 'INTC'),
            (     'Open', 'INTC'),
            (   'Volume', 'INTC')],
           names=['Price', 'Ticker'])
Flattened columns for INTC: Index(['INTC', 'INTC', 'INTC', 'INTC', 'INTC', 'INTC'], dtype='object', name='Ticker')

Processing data for AMT...


[*********************100%***********************]  1 of 1 completed


Original columns for AMT: MultiIndex([('Adj Close', 'AMT'),
            (    'Close', 'AMT'),
            (     'High', 'AMT'),
            (      'Low', 'AMT'),
            (     'Open', 'AMT'),
            (   'Volume', 'AMT')],
           names=['Price', 'Ticker'])
Flattened columns for AMT: Index(['AMT', 'AMT', 'AMT', 'AMT', 'AMT', 'AMT'], dtype='object', name='Ticker')

All stock data with technical indicators saved to /content/drive/My Drive/top_100_sp500_yfinance_data.csv.


In [15]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Load existing data
stock_data = pd.read_csv('/content/drive/My Drive/top_100_sp500_yfinance_data.csv', parse_dates=['Date'], index_col='Date')
stock_data.sort_index(inplace=True)  # Ensure data is sorted by date for time-based calculations

In [19]:
# Define function to get the last earnings date for each stock
def add_earnings_date(stock_subset, symbol):
    ticker = yf.Ticker(symbol)
    earnings_calendar = ticker.calendar

    # Check if earnings_calendar is a dict and contains 'Earnings Date'
    if isinstance(earnings_calendar, dict) and 'Earnings Date' in earnings_calendar:
        earnings_date = earnings_calendar['Earnings Date'][0]
    else:
        earnings_date = None

    stock_subset['earnings_date'] = earnings_date
    return stock_subset

# Apply to each stock in your dataset
for symbol in stock_data['Ticker'].unique():
    stock_subset = stock_data[stock_data['Ticker'] == symbol].copy()  # Make a copy
    stock_subset = add_earnings_date(stock_subset, symbol)
    stock_data.loc[stock_data['Ticker'] == symbol, 'earnings_date'] = stock_subset['earnings_date']


In [22]:
stock_data['price_momentum'] = stock_data.groupby('Ticker')['Adj Close'].transform(lambda x: x.diff(5))


In [23]:
import numpy as np

# Log return calculation
stock_data['log_return'] = stock_data.groupby('Ticker')['Adj Close'].apply(lambda x: np.log(x / x.shift(1)))
stock_data['historical_volatility'] = stock_data.groupby('Ticker')['log_return'].transform(lambda x: x.rolling(window=20).std())


In [24]:
stock_data['day_of_week'] = stock_data.index.to_series().apply(lambda x: x.weekday())
stock_data['month_of_year'] = stock_data.index.to_series().apply(lambda x: x.month)


In [25]:
# Save final dataset to CSV
stock_data.to_csv('/content/drive/My Drive/enriched_sp500_data.csv')

In [26]:
# Preview the final dataset
display(stock_data.head())

,Open,High,Low,Close,Adj Close,Volume,SMA_50,EMA_50,RSI_14,MACD_12_26_9,...,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Ticker,earnings_date,price_momentum,log_return,historical_volatility,day_of_week,month_of_year
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-02 00:00:00+00:00,176.665970,177.570007,177.779999,175.460007,175.520004,77334800,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AAPL,2025-01-30,NaN,NaN,NaN,3,11
2023-11-02 00:00:00+00:00,218.509995,218.509995,219.199997,211.449997,212.970001,125987600,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,TSLA,2025-01-24,NaN,NaN,NaN,3,11
2023-11-02 00:00:00+00:00,149.065125,150.850006,150.910004,148.330002,148.330002,3108400,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AXP,2025-01-24,NaN,NaN,NaN,3,11
2023-11-02 00:00:00+00:00,58.131916,59.910000,60.470001,58.500000,58.520000,15076500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NEE,2025-01-23,NaN,NaN,NaN,3,11
2023-11-02 00:00:00+00:00,77.986450,81.230003,81.309998,79.639999,79.809998,6166100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,GILD,2024-11-06,NaN,NaN,NaN,3,11


In [27]:
# Checking for NaNs in rolling window indicators
na_summary = stock_data[['SMA_50', 'EMA_50', 'RSI_14', 'MACD_12_26_9', 'BBU_20_2.0']].isna().sum()
print("Missing values in calculated fields due to window requirements:\n", na_summary)

# Determine the first non-NaN row for each indicator
non_na_start = stock_data[['SMA_50', 'EMA_50', 'RSI_14', 'MACD_12_26_9', 'BBU_20_2.0']].apply(lambda col: col.first_valid_index())
print("First non-NaN rows:\n", non_na_start)


Missing values in calculated fields due to window requirements:
 SMA_50          4851
EMA_50          4851
RSI_14          1386
MACD_12_26_9    2475
BBU_20_2.0      1881
dtype: int64
First non-NaN rows:
 SMA_50         2024-01-16 00:00:00+00:00
EMA_50         2024-01-16 00:00:00+00:00
RSI_14         2023-11-22 00:00:00+00:00
MACD_12_26_9   2023-12-08 00:00:00+00:00
BBU_20_2.0     2023-11-30 00:00:00+00:00
dtype: datetime64[ns, UTC]


In [28]:
# Define the start date for the complete dataset
start_date = "2024-01-16"

# Filter dataset to remove initial rows with NaNs due to rolling calculations
stock_data_filtered = stock_data[stock_data.index >= start_date]

# Verify that there are no NaNs in rolling window indicators
na_summary_filtered = stock_data_filtered[['SMA_50', 'EMA_50', 'RSI_14', 'MACD_12_26_9', 'BBU_20_2.0']].isna().sum()
print("NaN values after filtering:\n", na_summary_filtered)


NaN values after filtering:
 SMA_50          0
EMA_50          0
RSI_14          0
MACD_12_26_9    0
BBU_20_2.0      0
dtype: int64


In [29]:
from sklearn.preprocessing import MinMaxScaler

# Identify columns to normalize (excluding categorical and date-related columns)
numeric_columns = stock_data_filtered.select_dtypes(include=['float64', 'int64']).columns

# Initialize and fit the scaler
scaler = MinMaxScaler()
stock_data_filtered[numeric_columns] = scaler.fit_transform(stock_data_filtered[numeric_columns])

# Preview the normalized dataset
stock_data_filtered.head()


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
<ipython-input-29-16d29aecbdfd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data_filtered[numeric_columns] = scaler.fit_transform(stock_data_filtered[numeric_columns])


,Open,High,Low,Close,Adj Close,Volume,SMA_50,EMA_50,RSI_14,MACD_12_26_9,...,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Ticker,earnings_date,price_momentum,log_return,historical_volatility,day_of_week,month_of_year
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-16 00:00:00+00:00,0.003348,0.003387,0.003363,0.003413,0.003455,0.051079,0.003582,0.003538,0.380478,0.556006,...,0.003765,0.049288,0.191705,BAC,2025-01-16,0.547924,NaN,NaN,0.25,0.0
2024-01-16 00:00:00+00:00,0.017929,0.018092,0.018178,0.018358,0.018616,0.009165,0.022798,0.022521,0.260653,0.545121,...,0.023386,0.279820,0.389427,NKE,2024-12-19,0.551814,NaN,NaN,0.25,0.0
2024-01-16 00:00:00+00:00,0.016733,0.016856,0.016464,0.017008,0.016738,0.005159,0.018148,0.017928,0.739452,0.558369,...,0.017173,0.062219,0.731720,TJX,2024-11-20,0.553505,NaN,NaN,0.25,0.0
2024-01-16 00:00:00+00:00,0.057096,0.058102,0.057438,0.058895,0.058789,0.002434,0.065306,0.064512,0.515197,0.562719,...,0.061232,0.021998,0.481506,MCD,2024-10-29,0.555030,NaN,NaN,0.25,0.0
2024-01-16 00:00:00+00:00,0.004607,0.004902,0.004776,0.004816,0.004763,0.027106,0.005138,0.005076,0.524423,0.555737,...,0.004952,0.076631,0.562355,VZ,2025-01-24,0.548256,NaN,NaN,0.25,0.0


In [31]:
# Saving the processed and normalized dataset to a CSV file
final_csv_path = '/content/drive/My Drive/stock_data_normalized.csv'
stock_data_filtered.to_csv(final_csv_path)

final_csv_path

'/content/drive/My Drive/stock_data_normalized.csv'